# Synchronize homework files

In [1]:
! rm -rf sample_data; curl -s https://raw.githubusercontent.com/mposa/MEAM517/master/sync_hw.py > sync_hw.py
files = ["find_throwing_trajectory.py", "dynamics_constraints.py", "kinematic_constraints.py", "planar_arm.urdf"]

from sync_hw import sync_hw
sync_hw(5, files, False)

# Install Drake and set up a server for the visualizer

In [ ]:
# Install drake. We are using the lqr controller in drake as the final stabilizing controller.
# The installation process will take about 2 minutes but it's only required in the start of the Colab's virtual machine.
!curl -s https://raw.githubusercontent.com/mposa/MEAM517/master/colab_drake_setup.py > colab_drake_setup.py
from colab_drake_setup import setup
setup()

# Install pyngrok.
server_args = []
!pip install pyngrok
server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Problem 4 - Trajectory Optimization for a planar arm

## Compute the end effector (the ball) position and velocity using symbolic variables

This is how we get the expression in `kinematic_constraints.py`

In [ ]:
# Computes the end effector velocity as xdot, ydot J_q (pos) * qdot

from sympy import *
import numpy as np

l, q0, q1, q2, qdot0, qdot1, qdot2 = symbols('l q0 q1 q2 qdot0 qdot1 qdot2', real=True)
pos = l * Matrix([sin(q0), -cos(q0)]) + \
      l * Matrix([sin(q0 + q1), -cos(q0 + q1)]) + \
      l * Matrix([sin(q0 + q1 + q2), -cos(q0 + q1 + q2)])
vel = Matrix(pos).jacobian([q0, q1, q2]) @ np.reshape(np.array([qdot0, qdot1, qdot2]), (3,1))

print('pos (x_ball, z_ball):\n', pos)
print('')
print('vel (x_ball dot, z_ball dot):\n', vel)

## Run trajectory optimization to find the optimal trajectory

In [ ]:
import importlib
import find_throwing_trajectory
importlib.reload(find_throwing_trajectory)
from find_throwing_trajectory import find_throwing_trajectory

N = 7
initial_state = np.zeros(6)
tf = 5.0
distance = 25.0
x_traj, u_traj, prog, x_guess, u_guess = find_throwing_trajectory(N, initial_state, distance, tf)

## Visualize the optimal trajectory
Open the visualizer by clicking on the url in the output of this cell.

In [62]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import time
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.analysis import Simulator
from pydrake.systems.drawing import plot_system_graphviz

from pydrake.all import ( 
  ConnectMeshcatVisualizer, DiagramBuilder, 
  Simulator
)

from pydrake.geometry import (
  SceneGraph, ConnectDrakeVisualizer
)

from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant
from pydrake.multibody.parsing import Parser
from pydrake.systems.rendering import MultibodyPositionToGeometryPose
from pydrake.systems.primitives import (
  TrajectorySource,
  Demultiplexer,
  ConstantVectorSource
)

# Create a MultibodyPlant for the arm
file_name = "planar_arm.urdf"
builder = DiagramBuilder()
scene_graph = builder.AddSystem(SceneGraph())
planar_arm = builder.AddSystem(MultibodyPlant(0.0))
planar_arm.RegisterAsSourceForSceneGraph(scene_graph)
Parser(plant=planar_arm).AddModelFromFile(file_name)
planar_arm.Finalize()

# Create meshcat
visualizer = ConnectMeshcatVisualizer(
    builder, 
    scene_graph, 
    scene_graph.get_pose_bundle_output_port(),
    zmq_url=zmq_url,
    server_args=server_args)

x_traj_source = builder.AddSystem(TrajectorySource(x_traj))
u_traj_source = builder.AddSystem(TrajectorySource(u_traj))

demux = builder.AddSystem(Demultiplexer(np.array([3, 3])))
to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(planar_arm))
zero_inputs = builder.AddSystem(ConstantVectorSource(np.zeros(3)))

builder.Connect(zero_inputs.get_output_port(), planar_arm.get_actuation_input_port())
builder.Connect(x_traj_source.get_output_port(), demux.get_input_port())
builder.Connect(demux.get_output_port(0), to_pose.get_input_port())
builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(planar_arm.get_source_id()))

ConnectDrakeVisualizer(builder, scene_graph);

diagram = builder.Build()
diagram.set_name("diagram")

visualizer.load()
print("\n!!!Open the visualizer by clicking on the URL above!!!")

# Visualize the motion for `n_playback` times
n_playback = 3
for i in range(n_playback):
  # Set up a simulator to run this diagram.
  simulator = Simulator(diagram)
  simulator.Initialize()
  simulator.set_target_realtime_rate(1)
  simulator.AdvanceTo(tf);
  time.sleep(2)